# TP2 DATA831

Par RISS Ryan
IDU4

## Partie I : Installation

Pyspark + pandas

In [ ]:
pip install pyspark

In [ ]:
pip install pandas

## Partie II : Utilisation de PySpark

### Utilisation de fichiers .txt

On crée d'abord une configuration Spark et un contexte Spark.

In [110]:
from pyspark import SparkContext, SparkConf

In [111]:
conf = SparkConf()
# create Spark context with Spark configuration
sc = SparkContext.getOrCreate(conf=conf)

Puis on teste Spark avec un map reduce sur le fichier test.txt.

test.txt ayant pour contenu : *Je JE A; A ;*

On veut ici tester si spark détecte d'abord les différents mots et si il est sensible aux minuscules/majuscules et aux ponctuations.

In [112]:

 # read in text file and split each document into words
words = sc.textFile("data/test.txt").flatMap(lambda line: line.split(" "))
 # count the occurrence of each word
wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)

print(wordCounts.collect())

[('JE', 1), (';', 1), ('Je', 1), ('A;', 1), ('A', 1)]


Le map reduce de Spark considère un mot comme une suite de caractères attachées et est sensibles aux différence entre min/majuscules.

### Utilisation de CSVs

In [113]:
from pyspark.sql import SparkSession

spSession = SparkSession.builder.appName("Ok").getOrCreate()

On charge le csv, on afficher son schéma et puis on affiche les 10 premières entrées.

In [ ]:
df = spSession.read.csv("data/epl1.csv", header=True, inferSchema=True, sep = ';')

df.printSchema()

df.show(10)

## Partie III : Utilisation de pandas

On importe d'abord pandas et reduce()

In [114]:
import pandas as pd
from functools import reduce

On lis le csv

In [ ]:
df = pd.read_csv("data/epl1.csv", sep=";")

print(df)

### Nombre moyen de spectateurs par match

**Méthode normal**

In [116]:
df['attendance'].mean()

36451.836842105266

**Map reduce**

In [117]:
def mapper(x):
    return x

def reducer(acc, x):
    return acc + x

total = reduce(reducer, map(mapper, df['attendance']))

print(total/len(df))

36451.836842105266


On a le même résultat !

### Nombre moyen de buts par match

**Méthode classique**

In [118]:
sum = df['away_goals'].mean() + df['home_goals'].mean()

print("Resultat : " + str(sum/2))

Resultat : 1.35


**Map reduce**

In [119]:
sum = reduce(reducer, map(mapper, df['away_goals']))/len(df) + reduce(reducer, map(mapper, df['home_goals']))/len(df)

print("Resultat : " + str(sum/2))

Resultat : 1.35


### Nombre de buts pour chaque équipe

**Méthode classique**

In [120]:
print(df.groupby('away_team')['away_goals'].sum() + df.groupby('home_team')['home_goals'].sum())

away_team
Arsenal           65
Aston Villa       27
Bournemouth       45
Chelsea           59
Crystal Palace    39
Everton           59
Leicester         68
Liverpool         63
Man City          71
Man Utd           49
Newcastle         44
Norwich           39
Southampton       59
Spurs             69
Stoke             41
Sunderland        48
Swansea           42
Watford           40
West Brom         34
West Ham          65
dtype: int64


**Map reduce**

In [121]:
import collections

def mapperAway(row):
    return (row['away_team'], row['away_goals'])

def mapperHome(row):
    return (row['home_team'], row['home_goals'])


def reducer2(acc, x):
    team, goals = x
    acc[team] = acc.get(team, 0) + goals
    return acc

mp1 = reduce(reducer2, map(mapperAway, df.to_dict('records')), {})
mp2 = reduce(reducer2, map(mapperHome, df.to_dict('records')), {})

concatSeries = pd.concat([pd.Series(mp1), pd.Series(mp2)], axis=1)

concatSeries['total'] = concatSeries[0]+concatSeries[1]

print(concatSeries)

                 0   1  total
Bournemouth     22  23     45
Aston Villa     13  14     27
Leicester       33  35     68
Norwich         13  26     39
Spurs           34  35     69
Crystal Palace  20  19     39
West Ham        31  34     65
Man City        24  47     71
Sunderland      25  23     48
Liverpool       30  33     63
Chelsea         27  32     59
Watford         20  20     40
Everton         24  35     59
Man Utd         22  27     49
Arsenal         34  31     65
Southampton     20  39     59
Newcastle       12  32     44
West Brom       14  20     34
Stoke           19  22     41
Swansea         22  20     42
